In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

In [95]:
X_train=pd.read_csv('../input/month-dacon11/train_features.csv')
X_test=pd.read_csv('../input/month-dacon11/test_features.csv')
y_train=pd.read_csv('../input/month-dacon11/train_labels.csv')
submit=pd.read_csv('../input/month-dacon11/sample_submission.csv')

In [96]:
mean=X_train.iloc[:,2:].mean(axis=0)
std=X_train.iloc[:,2:].std(axis=0)

X_train_lstm=np.zeros((3125,600,6))
for uid in range(3125):
    df=X_train[X_train.id==uid].iloc[:,2:]
    X_train_lstm[uid,:,:]=(df-mean)/std
    
X_test_lstm=np.zeros((782,600,6))
for uid in range(782):
    df=X_test[X_test.id==uid+3125].iloc[:,2:]
    X_test_lstm[uid,:,:]=(df-mean)/std

In [97]:
from keras.utils import to_categorical

y=to_categorical(y_train['label'])

In [98]:
non_ex_idx=y_train[y_train['label_desc']=='Non-Exercise'].index
ex_idx=y_train[y_train['label_desc']!='Non-Exercise'].index

X_tr_ex=X_train_lstm[ex_idx,:,:]
y_tr_ex=y[ex_idx,:]

X_tr_non_ex=X_train_lstm[non_ex_idx,:,:]
y_tr_non_ex=y[non_ex_idx,:]

In [99]:
def aug(data,shift=0):
    shift_data=np.roll(data,shift,axis=1)
    return shift_data

X_train_aug=np.zeros((1607*12,600,6))
for i in range(12):
    aug_data=aug(X_tr_ex,int(np.random.random()*600))
    X_train_aug[1607*i:1607*(i+1),:,:]=aug_data
    
y_train_aug=np.vstack([y_tr_ex]*12)

In [100]:
X_tr_aug=np.vstack([X_tr_non_ex,X_train_aug])
y_tr_aug=np.vstack([y_tr_non_ex,y_train_aug])

## 1. Conv1D LSTM model

In [104]:
model = keras.Sequential([
keras.layers.Conv1D(filters=16,kernel_size=7,strides=1,padding='same',
                    activation='relu',input_shape=(600,6)),
keras.layers.BatchNormalization(),
keras.layers.MaxPooling1D(pool_size=2,strides=2),
keras.layers.Conv1D(filters=32,kernel_size=5,strides=1,
                                 padding='same',activation='relu'),
keras.layers.BatchNormalization(),
keras.layers.MaxPooling1D(pool_size=2,strides=2),
keras.layers.Conv1D(filters=64,kernel_size=3,strides=1,
                    padding='same',activation='relu'),
keras.layers.BatchNormalization(),
keras.layers.MaxPooling1D(pool_size=2,strides=2),
keras.layers.LSTM(180,return_sequences=True),
keras.layers.LSTM(180),
keras.layers.BatchNormalization(),
keras.layers.Dense(180,activation='relu'),
keras.layers.BatchNormalization(),
keras.layers.Dense(61,activation='softmax')])

model.compile(loss='categorical_crossentropy',
                     optimizer='rmsprop',
                     metrics =['categorical_crossentropy'])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_31 (Conv1D)           (None, 600, 16)           688       
_________________________________________________________________
batch_normalization_48 (Batc (None, 600, 16)           64        
_________________________________________________________________
max_pooling1d_30 (MaxPooling (None, 300, 16)           0         
_________________________________________________________________
conv1d_32 (Conv1D)           (None, 300, 32)           2592      
_________________________________________________________________
batch_normalization_49 (Batc (None, 300, 32)           128       
_________________________________________________________________
max_pooling1d_31 (MaxPooling (None, 150, 32)           0         
_________________________________________________________________
conv1d_33 (Conv1D)           (None, 150, 64)          

In [105]:
check=keras.callbacks.ModelCheckpoint('conv1d_lstm_1.h5',save_best_only=True)
earlystop=keras.callbacks.EarlyStopping(patience=5,restore_best_weights=True)
history=model.fit(X_tr_aug,y_tr_aug,epochs=10,
                    validation_split=0.2,verbose=1,batch_size=32,
                    callbacks=[check,earlystop])

Epoch 1/10
521/521 [==============================] - 14s 19ms/step - loss: 2.3909 - categorical_crossentropy: 2.3909 - val_loss: 1.3055 - val_categorical_crossentropy: 1.3055
Epoch 2/10
521/521 [==============================] - 9s 17ms/step - loss: 1.1952 - categorical_crossentropy: 1.1952 - val_loss: 0.7717 - val_categorical_crossentropy: 0.7717
Epoch 3/10
521/521 [==============================] - 9s 18ms/step - loss: 0.8667 - categorical_crossentropy: 0.8667 - val_loss: 0.6608 - val_categorical_crossentropy: 0.6608
Epoch 4/10
521/521 [==============================] - 9s 18ms/step - loss: 0.6657 - categorical_crossentropy: 0.6657 - val_loss: 0.4639 - val_categorical_crossentropy: 0.4639
Epoch 5/10
521/521 [==============================] - 9s 17ms/step - loss: 0.5308 - categorical_crossentropy: 0.5308 - val_loss: 0.3477 - val_categorical_crossentropy: 0.3477
Epoch 6/10
521/521 [==============================] - 9s 18ms/step - loss: 0.4618 - categorical_crossentropy: 0.4618 - val_l

In [106]:
pred=model.predict(X_test_lstm)

In [109]:
submit.iloc[:,1:]+=pred
submit.head()

,id,0,1,2,3,4,5,6,7,8,...,51,52,53,54,55,56,57,58,59,60
0,3125,1.049403e-01,0.003157,4.294519e-03,4.174639e-04,1.923851e-04,7.244886e-05,0.000045,4.497101e-04,1.741927e-03,...,1.452662e-03,6.629559e-05,0.000064,6.201591e-05,2.022727e-03,3.981476e-03,1.027448e-05,1.180717e-03,4.405346e-04,6.785927e-04
1,3126,4.821738e-06,0.000012,1.665145e-06,3.098293e-05,3.932365e-07,1.444088e-05,0.000016,9.275016e-06,3.610634e-05,...,2.163644e-05,2.739548e-05,0.000074,3.891872e-06,5.321477e-06,1.287929e-07,4.792363e-07,7.741846e-06,7.830334e-08,1.886858e-05
2,3127,2.228672e-09,0.000006,1.041107e-07,2.498027e-06,1.165300e-07,4.614803e-07,0.000004,1.775803e-07,7.427315e-08,...,8.780445e-07,5.135947e-08,0.000003,1.740506e-06,1.000317e-07,4.899388e-09,1.299201e-07,2.786279e-08,5.650910e-08,1.983479e-07
3,3128,3.178996e-05,0.000003,1.430139e-05,6.760331e-06,3.071855e-07,9.895998e-06,0.000015,1.069729e-06,3.475610e-05,...,8.694272e-06,4.587162e-05,0.000006,1.288210e-05,4.660895e-06,9.757606e-07,3.205124e-07,6.952068e-06,9.455123e-07,5.534193e-05
4,3129,9.656593e-06,0.000004,3.977859e-06,5.909888e-07,4.030716e-07,7.830168e-06,0.000013,5.495975e-07,1.318876e-04,...,9.258233e-06,2.077000e-06,0.000004,2.512836e-07,3.524581e-06,6.429221e-07,2.847746e-03,3.724756e-06,4.658567e-07,2.732779e-05


In [ ]:
submit.to_csv('20210219_conv1d_lstm_1.csv',index=False)